In [ ]:
## Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt


In [ ]:
# Comparison of ResNets and Wide-ResNets
#
# | Feature        | ResNet                                  | Wide ResNet                             |
# |----------------|------------------------------------------|-----------------------------------------|
# | Architecture   | Uses residual blocks with skip connections | Uses wider residual blocks              |
# | Width          | Relatively narrow                        | Increased width of convolutional layers |
# | Depth          | Can be very deep                         | Can be relatively shallower              |
# | Computational Cost | Lower                                  | Higher                                 |
# | Accuracy       | High, but can plateau with increasing depth | Often achieves higher accuracy for similar depth|
# | Memory Usage   | Lower                                  | Higher                                 |
# | Training Time  | Lower                                  | Higher                                 |
# | Advantages     | Efficient, good performance              | Higher accuracy, potentially better generalization |
# | Disadvantages  | Accuracy may saturate at very high depths| Higher computational and memory overhead |
# | Use Cases      | General purpose image classification     | Applications requiring high accuracy with limited depth |

## ResNets

In [ ]:
# Define a Basic Residual Block (Used in both ResNet and Wide-ResNet)
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity
        out = self.relu(out)
        return out

In [ ]:
# Standard ResNet (Narrow and Deep)
class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.layer1 = self._make_layer(3, 64, 2)
        self.layer2 = self._make_layer(64, 128, 2, stride=2)
        self.layer3 = self._make_layer(128, 256, 2, stride=2)
        self.layer4 = self._make_layer(256, 512, 2, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, in_channels, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        layers = [ResidualBlock(in_channels, out_channels, stride, downsample)]
        for _ in range(1, blocks):
            layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

## Wide-ResNets

In [ ]:
# Wide ResNet (Wider instead of Deeper)
class WideResNet(nn.Module):
    def __init__(self, num_classes=10, width_factor=2):  # Width factor increases channels
        super(WideResNet, self).__init__()
        self.layer1 = self._make_layer(3, 64 * width_factor, 2)
        self.layer2 = self._make_layer(64 * width_factor, 128 * width_factor, 2, stride=2)
        self.layer3 = self._make_layer(128 * width_factor, 256 * width_factor, 2, stride=2)
        self.layer4 = self._make_layer(256 * width_factor, 512 * width_factor, 2, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * width_factor, num_classes)

    def _make_layer(self, in_channels, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        layers = [ResidualBlock(in_channels, out_channels, stride, downsample)]
        for _ in range(1, blocks):
            layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [ ]:

from torchsummary import summary
import pandas as pd

# Assuming you have defined your models (ResNet and WideResNet) as in the previous response.

# Create instances of your models
resnet_model = ResNet()
wideresnet_model = WideResNet()

# Get summaries for both models
resnet_summary = summary(resnet_model, (3, 32, 32), device='cpu') # Replace with actual input shape
wideresnet_summary = summary(wideresnet_model, (3, 32, 32), device='cpu')

# Extract the relevant information
resnet_summary_str = str(resnet_summary)
wideresnet_summary_str = str(wideresnet_summary)


# Function to extract layer details
def extract_layer_info(summary_str):
    lines = summary_str.splitlines()
    header_index = next((i for i, line in enumerate(lines) if 'Output Shape' in line), None)
    if header_index is None:
        return []  # Or handle the error appropriately

    layer_info = []
    for line in lines[header_index + 1:]:
        parts = line.split()
        if len(parts) > 3: # Check to avoid blank lines
          layer_info.append([parts[0], parts[2]])

    return layer_info

# Extract layer information for each model
resnet_layers = extract_layer_info(resnet_summary_str)
wideresnet_layers = extract_layer_info(wideresnet_summary_str)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]             192
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]           1,728
       BatchNorm2d-4           [-1, 64, 32, 32]             128
              ReLU-5           [-1, 64, 32, 32]               0
            Conv2d-6           [-1, 64, 32, 32]          36,864
       BatchNorm2d-7           [-1, 64, 32, 32]             128
              ReLU-8           [-1, 64, 32, 32]               0
     ResidualBlock-9           [-1, 64, 32, 32]               0
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
             ReLU-12           [-1, 64, 32, 32]               0
           Conv2d-13           [-1, 64, 32, 32]          36,864
      BatchNorm2d-14           [-1, 64,